In [1]:
from llama_cpp import Llama
from rich import print
from langchain_core.utils.function_calling import convert_to_openai_function
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

In [2]:
llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.2-GGUF",
    filename="functionary-small-v2.2.q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.2-GGUF"),
    n_gpu_layers=-1,
    verbose=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from agents.tools import get_current_weather

oai_tool = convert_to_openai_function(get_current_weather)
print(oai_tool)

{
    'name': 'get_current_weather',
    'description': 'get_current_weather(city: str) -> str - Get the current weather for a given city.\n\n    
Args:\n      city (str): The city to fetch weather for.\n\n    Returns:\n      str: current weather condition, or 
None if an error occurs.',
    'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}
}

In [6]:
output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "How is the weather in London today?"}],
    tools=[{"type": "function", "function": oai_tool}],
)

print(output)

{
    'id': 'chatcmpl-339bfd4b-f22c-45a2-9265-40906eadafa9',
    'object': 'chat.completion',
    'created': 1715032029,
    'model': 
'/Users/aniket/.cache/huggingface/hub/models--meetkai--functionary-small-v2.2-GGUF/snapshots/9971ff5d1eacbfac81fe69
39bccd59cf7d668078/./functionary-small-v2.2.q4_0.gguf',
    'choices': [
        {
            'index': 0,
            'logprobs': None,
            'message': {
                'role': 'assistant',
                'content': None,
                'tool_calls': [
                    {
                        'id': 'call_YKQmVTO2Alwas834Yzf1UpJp',
                        'type': 'function',
                        'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}
                    }
                ]
            },
            'finish_reason': 'tool_calls'
        }
    ],
    'usage': {'prompt_tokens': 200, 'completion_tokens': 16, 'total_tokens': 201}
}

In [22]:
output

{'id': 'chatcmpl-339bfd4b-f22c-45a2-9265-40906eadafa9',
 'object': 'chat.completion',
 'created': 1715032029,
 'model': '/Users/aniket/.cache/huggingface/hub/models--meetkai--functionary-small-v2.2-GGUF/snapshots/9971ff5d1eacbfac81fe6939bccd59cf7d668078/./functionary-small-v2.2.q4_0.gguf',
 'choices': [{'index': 0,
   'logprobs': None,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_YKQmVTO2Alwas834Yzf1UpJp',
      'type': 'function',
      'function': {'name': 'get_current_weather',
       'arguments': '{"city": "London"}'}}]},
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 200, 'completion_tokens': 16, 'total_tokens': 201}}

In [19]:
c = ChatCompletion(**output)